In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models,initializers
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import random
#random.seed(40)
np.random.seed(0)
tf.random.set_seed(0)

In [2]:
# 定义一个简单的神经网络回归模型
def create_model(input_shape):
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    model.add(layers.Dense(100, activation='relu',kernel_initializer=initializers.HeNormal(),kernel_regularizer=tf.keras.regularizers.l2(0.01)))#隐藏层1
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(50, activation='relu',kernel_initializer=initializers.HeNormal(),kernel_regularizer=tf.keras.regularizers.l2(0.01)))#隐藏层2
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(10, activation='relu',kernel_initializer=initializers.HeNormal(),kernel_regularizer=tf.keras.regularizers.l2(0.01)))#隐藏层2
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1))  # 输出层为1个节点，无激活函数（线性）
    return model

In [3]:
#数值转换
def replace_values(val):
    if val == '60-70':
        return 65
    elif val == '71-80':
        return 75
    elif val == '>80':
        return 85
    else:
        return val

In [4]:
# 导入数据
data = pd.read_csv('训练集2.csv')

# 数据处理
data = data.drop(["Sl 2"], axis=1)#删除无意义
data = data.sample(frac=1)
data.reset_index(inplace=True)
data.drop(["index"], axis=1, inplace=True)#去掉索引

In [5]:
# 删除缺失率过高的变量
def filter_nan(data, narate = 0.2):
    n_samples = data.shape[0]
    list_nan_cols = []
    
    for col in data.columns:
        if data[col].isna().sum() / n_samples >= (narate):
            list_nan_cols.append(col)
    
    print(f'缺失量在{narate * 100}%以上的变量有:{list_nan_cols}')

    return list_nan_cols

In [6]:
list_nullfactor_todrop = filter_nan(data, narate=0.2)
data.drop(list_nullfactor_todrop, axis = 1,inplace=True)

缺失量在20.0%以上的变量有:['RR', 'HbA1c (%) ', 'Procalcitonin (ng/mL) ', 'IL-6 (pg/mL) ']


In [7]:
# 处理缺失值
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler

# Select numerical columns (excluding non-numeric and identifiers like 'Patient Code')
numerical_cols = data.select_dtypes(include=['float64', 'int64']).columns

# Standardize the numerical columns
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data[numerical_cols])

# Apply KNN Imputer
knn_imputer = KNNImputer(n_neighbors=10)
imputed_data = knn_imputer.fit_transform(scaled_data)

# Inverse transform to get the imputed values back to the original scale
imputed_data = scaler.inverse_transform(imputed_data)

# Update the original dataframe with the imputed values
data[numerical_cols] = imputed_data
#非数值处理
data['Age'] = data['Age'].apply(replace_values)

In [8]:
# 目标列和特征列
X = data.drop('outcome', axis=1)  # 特征集
y = data['outcome']  # 目标变量

In [9]:
# 数据标准化处理
scaler1 = MinMaxScaler()
X = scaler1.fit_transform(X)

scaler2 = StandardScaler()
X = scaler2.fit_transform(X)

In [10]:
# 将数据集分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)#, random_state=42)

In [11]:
# 设置超参数
input_shape = (X_train.shape[1],)  # 自动获取特征数量
batch_size = 64
num_epochs = 1000

In [12]:
# 构建模型
model = create_model(input_shape)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)  # 这里将学习率设置为0.001
# 编译模型
model.compile(optimizer=optimizer,
              loss='mean_squared_error',  # 使用均方误差作为损失函数
              metrics=[tf.keras.metrics.MeanAbsoluteError(), 
                       tf.keras.metrics.RootMeanSquaredError()])

# 训练模型
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test, y_test),callbacks=[early_stopping])

Epoch 1/1000
3/3 [==============================] - 1s 158ms/step - loss: 7.5723 - mean_absolute_error: 1.6327 - root_mean_squared_error: 2.0899 - val_loss: 4.9833 - val_mean_absolute_error: 1.1560 - val_root_mean_squared_error: 1.3381
Epoch 2/1000
3/3 [==============================] - 0s 16ms/step - loss: 6.8326 - mean_absolute_error: 1.4679 - root_mean_squared_error: 1.9079 - val_loss: 4.6255 - val_mean_absolute_error: 1.0271 - val_root_mean_squared_error: 1.2022
Epoch 3/1000
3/3 [==============================] - 0s 27ms/step - loss: 6.1945 - mean_absolute_error: 1.3205 - root_mean_squared_error: 1.7362 - val_loss: 4.4238 - val_mean_absolute_error: 0.9463 - val_root_mean_squared_error: 1.1205
Epoch 4/1000
3/3 [==============================] - 0s 20ms/step - loss: 6.4470 - mean_absolute_error: 1.3590 - root_mean_squared_error: 1.8111 - val_loss: 4.3354 - val_mean_absolute_error: 0.9222 - val_root_mean_squared_error: 1.0858
Epoch 5/1000
3/3 [==============================] - 0s 23ms

Epoch 36/1000
3/3 [==============================] - 0s 39ms/step - loss: 3.6400 - mean_absolute_error: 0.7047 - root_mean_squared_error: 0.9040 - val_loss: 3.5039 - val_mean_absolute_error: 0.6912 - val_root_mean_squared_error: 0.8306
Epoch 37/1000
3/3 [==============================] - 0s 39ms/step - loss: 3.6463 - mean_absolute_error: 0.7386 - root_mean_squared_error: 0.9133 - val_loss: 3.4835 - val_mean_absolute_error: 0.6842 - val_root_mean_squared_error: 0.8247
Epoch 38/1000
3/3 [==============================] - 0s 31ms/step - loss: 3.5288 - mean_absolute_error: 0.6780 - root_mean_squared_error: 0.8526 - val_loss: 3.4623 - val_mean_absolute_error: 0.6767 - val_root_mean_squared_error: 0.8182
Epoch 39/1000
3/3 [==============================] - 0s 39ms/step - loss: 3.6078 - mean_absolute_error: 0.7032 - root_mean_squared_error: 0.9037 - val_loss: 3.4386 - val_mean_absolute_error: 0.6672 - val_root_mean_squared_error: 0.8101
Epoch 40/1000
3/3 [==============================] - 0s 

3/3 [==============================] - 0s 47ms/step - loss: 3.1221 - mean_absolute_error: 0.5625 - root_mean_squared_error: 0.8078 - val_loss: 3.0168 - val_mean_absolute_error: 0.5929 - val_root_mean_squared_error: 0.7460
Epoch 71/1000
3/3 [==============================] - 0s 39ms/step - loss: 2.9726 - mean_absolute_error: 0.5511 - root_mean_squared_error: 0.7164 - val_loss: 3.0086 - val_mean_absolute_error: 0.5931 - val_root_mean_squared_error: 0.7469
Epoch 72/1000
3/3 [==============================] - 0s 39ms/step - loss: 3.0836 - mean_absolute_error: 0.6168 - root_mean_squared_error: 0.7963 - val_loss: 2.9967 - val_mean_absolute_error: 0.5903 - val_root_mean_squared_error: 0.7454
Epoch 73/1000
3/3 [==============================] - 0s 39ms/step - loss: 2.9525 - mean_absolute_error: 0.5236 - root_mean_squared_error: 0.7163 - val_loss: 2.9768 - val_mean_absolute_error: 0.5822 - val_root_mean_squared_error: 0.7383
Epoch 74/1000
3/3 [==============================] - 0s 50ms/step - lo

Epoch 105/1000
3/3 [==============================] - 0s 47ms/step - loss: 2.5035 - mean_absolute_error: 0.4739 - root_mean_squared_error: 0.6016 - val_loss: 2.5377 - val_mean_absolute_error: 0.4827 - val_root_mean_squared_error: 0.6350
Epoch 106/1000
3/3 [==============================] - 0s 55ms/step - loss: 2.4687 - mean_absolute_error: 0.4480 - root_mean_squared_error: 0.5794 - val_loss: 2.5181 - val_mean_absolute_error: 0.4744 - val_root_mean_squared_error: 0.6263
Epoch 107/1000
3/3 [==============================] - 0s 39ms/step - loss: 2.4807 - mean_absolute_error: 0.4597 - root_mean_squared_error: 0.5959 - val_loss: 2.4997 - val_mean_absolute_error: 0.4669 - val_root_mean_squared_error: 0.6183
Epoch 108/1000
3/3 [==============================] - 0s 31ms/step - loss: 2.4173 - mean_absolute_error: 0.4136 - root_mean_squared_error: 0.5491 - val_loss: 2.4851 - val_mean_absolute_error: 0.4621 - val_root_mean_squared_error: 0.6135
Epoch 109/1000
3/3 [==============================] 

3/3 [==============================] - 0s 55ms/step - loss: 2.1060 - mean_absolute_error: 0.3774 - root_mean_squared_error: 0.5045 - val_loss: 2.1929 - val_mean_absolute_error: 0.4428 - val_root_mean_squared_error: 0.5901
Epoch 140/1000
3/3 [==============================] - 0s 39ms/step - loss: 2.1416 - mean_absolute_error: 0.4186 - root_mean_squared_error: 0.5460 - val_loss: 2.1841 - val_mean_absolute_error: 0.4426 - val_root_mean_squared_error: 0.5894
Epoch 141/1000
3/3 [==============================] - 0s 39ms/step - loss: 2.1014 - mean_absolute_error: 0.4016 - root_mean_squared_error: 0.5159 - val_loss: 2.1741 - val_mean_absolute_error: 0.4416 - val_root_mean_squared_error: 0.5877
Epoch 142/1000
3/3 [==============================] - 0s 48ms/step - loss: 2.1450 - mean_absolute_error: 0.4216 - root_mean_squared_error: 0.5635 - val_loss: 2.1649 - val_mean_absolute_error: 0.4412 - val_root_mean_squared_error: 0.5866
Epoch 143/1000
3/3 [==============================] - 0s 53ms/step 

...

3/3 [==============================] - 0s 32ms/step - loss: 0.4805 - mean_absolute_error: 0.2379 - root_mean_squared_error: 0.3147 - val_loss: 0.6146 - val_mean_absolute_error: 0.3888 - val_root_mean_squared_error: 0.4842
Epoch 482/1000
3/3 [==============================] - 0s 44ms/step - loss: 0.5314 - mean_absolute_error: 0.2532 - root_mean_squared_error: 0.3881 - val_loss: 0.6126 - val_mean_absolute_error: 0.3888 - val_root_mean_squared_error: 0.4836
Epoch 483/1000
3/3 [==============================] - 0s 47ms/step - loss: 0.4967 - mean_absolute_error: 0.2385 - root_mean_squared_error: 0.3438 - val_loss: 0.6112 - val_mean_absolute_error: 0.3891 - val_root_mean_squared_error: 0.4837
Epoch 484/1000
3/3 [==============================] - 0s 42ms/step - loss: 0.4572 - mean_absolute_error: 0.2058 - root_mean_squared_error: 0.2831 - val_loss: 0.6102 - val_mean_absolute_error: 0.3892 - val_root_mean_squared_error: 0.4842
Epoch 485/1000
3/3 [==============================] - 0s 40ms/step 

Epoch 515/1000
3/3 [==============================] - 0s 39ms/step - loss: 0.4224 - mean_absolute_error: 0.2110 - root_mean_squared_error: 0.3037 - val_loss: 0.5710 - val_mean_absolute_error: 0.3926 - val_root_mean_squared_error: 0.4922
Epoch 516/1000
3/3 [==============================] - 0s 37ms/step - loss: 0.4156 - mean_absolute_error: 0.2094 - root_mean_squared_error: 0.2938 - val_loss: 0.5675 - val_mean_absolute_error: 0.3924 - val_root_mean_squared_error: 0.4901
Epoch 517/1000
3/3 [==============================] - 0s 39ms/step - loss: 0.4210 - mean_absolute_error: 0.2169 - root_mean_squared_error: 0.3064 - val_loss: 0.5643 - val_mean_absolute_error: 0.3921 - val_root_mean_squared_error: 0.4883
Epoch 518/1000
3/3 [==============================] - 0s 40ms/step - loss: 0.4354 - mean_absolute_error: 0.2245 - root_mean_squared_error: 0.3293 - val_loss: 0.5626 - val_mean_absolute_error: 0.3925 - val_root_mean_squared_error: 0.4880
Epoch 519/1000
3/3 [==============================] 

3/3 [==============================] - 0s 45ms/step - loss: 0.3674 - mean_absolute_error: 0.2046 - root_mean_squared_error: 0.2829 - val_loss: 0.5308 - val_mean_absolute_error: 0.3953 - val_root_mean_squared_error: 0.4948
Epoch 550/1000
3/3 [==============================] - 0s 42ms/step - loss: 0.3711 - mean_absolute_error: 0.1979 - root_mean_squared_error: 0.2918 - val_loss: 0.5300 - val_mean_absolute_error: 0.3954 - val_root_mean_squared_error: 0.4952
Epoch 551/1000
3/3 [==============================] - ETA: 0s - loss: 0.3883 - mean_absolute_error: 0.2109 - root_mean_squared_error: 0.321 - 0s 32ms/step - loss: 0.3774 - mean_absolute_error: 0.2049 - root_mean_squared_error: 0.3043 - val_loss: 0.5282 - val_mean_absolute_error: 0.3955 - val_root_mean_squared_error: 0.4943
Epoch 552/1000
3/3 [==============================] - 0s 43ms/step - loss: 0.3490 - mean_absolute_error: 0.1885 - root_mean_squared_error: 0.2557 - val_loss: 0.5264 - val_mean_absolute_error: 0.3963 - val_root_mean_s

3/3 [==============================] - 0s 39ms/step - loss: 0.3458 - mean_absolute_error: 0.2178 - root_mean_squared_error: 0.3095 - val_loss: 0.4936 - val_mean_absolute_error: 0.3961 - val_root_mean_squared_error: 0.4944
Epoch 584/1000
3/3 [==============================] - ETA: 0s - loss: 0.3394 - mean_absolute_error: 0.1981 - root_mean_squared_error: 0.300 - 0s 88ms/step - loss: 0.3337 - mean_absolute_error: 0.1933 - root_mean_squared_error: 0.2912 - val_loss: 0.4918 - val_mean_absolute_error: 0.3974 - val_root_mean_squared_error: 0.4936
Epoch 585/1000
3/3 [==============================] - 0s 38ms/step - loss: 0.3150 - mean_absolute_error: 0.1895 - root_mean_squared_error: 0.2587 - val_loss: 0.4910 - val_mean_absolute_error: 0.3989 - val_root_mean_squared_error: 0.4937
Epoch 586/1000
3/3 [==============================] - 0s 39ms/step - loss: 0.3185 - mean_absolute_error: 0.1973 - root_mean_squared_error: 0.2671 - val_loss: 0.4917 - val_mean_absolute_error: 0.4007 - val_root_mean_s

Epoch 617/1000
3/3 [==============================] - 0s 39ms/step - loss: 0.2723 - mean_absolute_error: 0.1610 - root_mean_squared_error: 0.2297 - val_loss: 0.4607 - val_mean_absolute_error: 0.3967 - val_root_mean_squared_error: 0.4923
Epoch 618/1000
3/3 [==============================] - 0s 31ms/step - loss: 0.2823 - mean_absolute_error: 0.1715 - root_mean_squared_error: 0.2532 - val_loss: 0.4604 - val_mean_absolute_error: 0.3971 - val_root_mean_squared_error: 0.4928
Epoch 619/1000
3/3 [==============================] - 0s 40ms/step - loss: 0.3164 - mean_absolute_error: 0.2118 - root_mean_squared_error: 0.3147 - val_loss: 0.4599 - val_mean_absolute_error: 0.3969 - val_root_mean_squared_error: 0.4933
Epoch 620/1000
3/3 [==============================] - 0s 39ms/step - loss: 0.2916 - mean_absolute_error: 0.1881 - root_mean_squared_error: 0.2739 - val_loss: 0.4583 - val_mean_absolute_error: 0.3956 - val_root_mean_squared_error: 0.4924
Epoch 621/1000
3/3 [==============================] 

In [13]:
# 测试模型
test_loss, test_mae, test_rmse = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest Mean Absolute Error (MAE): {test_mae:.2f}')
print(f'Test Root Mean Squared Error (RMSE): {test_rmse:.2f}')

# 进行预测
y_pred = model.predict(X_test)

# 计算额外的MAE和RMSE指标
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'Calculated Mean Absolute Error (MAE): {mae:.2f}')
print(f'Calculated Root Mean Squared Error (RMSE): {rmse:.2f}')

2/2 - 0s - loss: 0.4488 - mean_absolute_error: 0.3986 - root_mean_squared_error: 0.4870

Test Mean Absolute Error (MAE): 0.40
Test Root Mean Squared Error (RMSE): 0.49
Calculated Mean Absolute Error (MAE): 0.40
Calculated Root Mean Squared Error (RMSE): 0.49
